In [4]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom
import numpy as np
import os
import lsst.utils
import pfs.drp.stella.findAndTraceAperturesTask as fataTask
import pfs.drp.stella as drpStella
try:
    import lsst.afw.display as afwDisplay
except ImportError:
    afwDisplay = None

if afwDisplay:
    try:
        afwDisplay.setDefaultBackend("ds9" if True else "virtualDevice")
    except RuntimeError as e:
        print e

    afwDisplay.setDefaultMaskTransparency(75)

In [5]:
testDataDir = os.path.join(lsst.utils.getPackageDir("drp_stella_data"),'data/PFS')
butler = dafPersist.Butler(testDataDir)

type(config) =  <class 'lsst.afw.cameraGeom.cameraConfig.CameraConfig'>
type(config) =  <class 'lsst.afw.cameraGeom.cameraConfig.CameraConfig'>


In [6]:
dataId = dict(site='S', category='A', filter='r', visit=5, ccd=5)
exposure = butler.get('postISRCCD', dataId)
print exposure

RuntimeError: No unique lookup for ['taiObs', 'dateObs', 'field', 'spectrograph', 'arm'] from {'filter': 'r', 'ccd': 5, 'category': 'A', 'visit': 5, 'site': 'S'}: 0 matches

In [4]:
display0 = afwDisplay.getDisplay()
display0.mtv(exposure, title="parent")
display0.setMaskTransparency(50)
display0.setMaskPlaneColor("CROSSTALK", "orange")

In [5]:
print dir(exposure)

['Factory', '__class__', '__del__', '__delattr__', '__dict__', '__doc__', '__float__', '__format__', '__getattr__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__int__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__swig_getmethods__', '__swig_setmethods__', '__weakref__', '_s', 'census', 'clone', 'getBBox', 'getCalib', 'getDetector', 'getDimensions', 'getFilter', 'getHeight', 'getId', 'getInfo', 'getMaskedImage', 'getMetadata', 'getNextMemId', 'getPsf', 'getWcs', 'getWidth', 'getX0', 'getXY0', 'getY0', 'hasBeenCorrupted', 'hasPsf', 'hasWcs', 'init', 'magicSentinel', 'markPersistent', 'readFits', 'repr', 'setCalib', 'setCorruptionCallback', 'setDeleteCallback', 'setDeleteCallbackId', 'setDetector', 'setFilter', 'setMaskedImage', 'setMetadata', 'setNewCallback', 'setNewCallbackId', 'setPsf', 'setWcs', 'setXY0', 'swigConvert', 'this', 'writeFits']


In [6]:
myFindTask = fataTask.FindAndTraceAperturesTask()
fts = myFindTask.run(exposure)
print fts.size()

inMaskedImage created
fts.size() =  11
11


In [7]:
for iTrace in range(fts.size()):
    ft = fts.getFiberTrace(iTrace)
    xCenters = ft.getXCenters()
    ftFunction = ft.getFiberTraceFunction()
    yLow = ftFunction.yCenter + ftFunction.yLow
    yHigh = ftFunction.yCenter + ftFunction.yHigh

    for y in np.arange(yLow, yHigh+1, 20):
        point = afwGeom.Point2D(xCenters[y-yLow],y)
        display0.dot("+", xCenters[y-yLow], y)

In [10]:
for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, exposure.getMaskedImage())

In [11]:
display0 = afwDisplay.getDisplay()
display0.mtv(exposure, title="parent")
display0.setMaskTransparency(50)
display0.setMaskPlaneColor("CROSSTALK", "orange")